# Preprocessing datasets

File dedicated to preprocessing datasets to preprare them for use in the final model. 

In [1]:
# importing packages
import pandas as pd
import os
import problem

## US GDP data

In [3]:
us_gdp = pd.read_csv(os.path.join('raw_add_data', 'gdp_data.csv'), na_values='(D)')
us_gdp.head()

,GeoFips,GeoName,LineCode,Description,2011,2012,2013
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",1.0,All industry total,277405509.0,287570424.0,299918186.0
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2.0,Private industries,250779823.0,260962774.0,273113553.0
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3.0,"Agriculture, forestry, fishing and hunting",NaN,NaN,602096.0
3,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",6.0,"Mining, quarrying, and oil and gas extraction",267771.0,228417.0,231368.0
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",10.0,Utilities,3704264.0,3582854.0,NaN


The first step is to associate the column GeoName with the list of airports from the original problem. 

In [4]:
cities = us_gdp['GeoName'].unique()
print(f'Total cities: {len(cities)}')
print(f'The cities are: \n{cities}')


Total cities: 18
The cities are: 
['Atlanta-Sandy Springs-Alpharetta, GA (Metropolitan Statistical Area)'
 'Austin-Round Rock-Georgetown, TX (Metropolitan Statistical Area)'
 'Boston-Cambridge-Newton, MA-NH (Metropolitan Statistical Area)'
 'Charlotte-Concord-Gastonia, NC-SC (Metropolitan Statistical Area)'
 'Chicago-Naperville-Elgin, IL-IN-WI (Metropolitan Statistical Area)'
 'Denver-Aurora-Lakewood, CO (Metropolitan Statistical Area) *'
 'Detroit-Warren-Dearborn, MI (Metropolitan Statistical Area)'
 'Houston-The Woodlands-Sugar Land, TX (Metropolitan Statistical Area)'
 'Las Vegas-Henderson-Paradise, NV (Metropolitan Statistical Area)'
 'Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)'
 'Miami-Fort Lauderdale-Pompano Beach, FL (Metropolitan Statistical Area)'
 'Minneapolis-St. Paul-Bloomington, MN-WI (Metropolitan Statistical Area)'
 'New York-Newark-Jersey City, NY-NJ-PA (Metropolitan Statistical Area)'
 'Orlando-Kissimmee-Sanford, FL (Metropolitan Statistical Are

In [5]:
#retrieve name of the first city from GeoName column
us_gdp['city'] = us_gdp['GeoName'].apply(lambda city: city.split('-')[0])
cities = us_gdp['city'].unique()
print('The values in the column city are:\n', cities) 

The values in the column city are:
 ['Atlanta' 'Austin' 'Boston' 'Charlotte' 'Chicago' 'Denver' 'Detroit'
 'Houston' 'Las Vegas' 'Los Angeles' 'Miami' 'Minneapolis' 'New York'
 'Orlando' 'Philadelphia' 'Phoenix' 'San Francisco' 'Seattle']


We can check the correspondance between the cities and the airports from the file `city_ariport.csv`

In [7]:
city_airport = pd.read_csv(os.path.join('raw_add_data', 'city_airport.csv'), header=None)
city_airport.columns = ['airport', 'city']
print(city_airport)

   airport           city
0      ATL        Atlanta
1      BOS         Boston
2      CLT      Charlotte
3      ORD        Chicago
4      DEN         Denver
5      DTW        Detroit
6      DFW         Austin
7      IAH        Houston
8      LAS      Las Vegas
9      LAX    Los Angeles
10     MIA          Miami
11     MSP    Minneapolis
12     JFK       New York
13     LGA       New York
14     EWR       New York
15     MCO        Orlando
16     PHL   Philadelphia
17     PHX        Phoenix
18     SFO  San Francisco
19     SEA        Seattle


In [8]:
merged_us_gdp = pd.merge(us_gdp, city_airport, on='city')
merged_us_gdp.head()

,GeoFips,GeoName,LineCode,Description,2011,2012,2013,city,airport
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",1.0,All industry total,277405509.0,287570424.0,299918186.0,Atlanta,ATL
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2.0,Private industries,250779823.0,260962774.0,273113553.0,Atlanta,ATL
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3.0,"Agriculture, forestry, fishing and hunting",NaN,NaN,602096.0,Atlanta,ATL
3,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",6.0,"Mining, quarrying, and oil and gas extraction",267771.0,228417.0,231368.0,Atlanta,ATL
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",10.0,Utilities,3704264.0,3582854.0,NaN,Atlanta,ATL


The columns `city`, `GeoFips` and `GeoName` are no longer useful. The column `Description` is encode in `LineCode` and can also be deleted.

In [9]:
del (merged_us_gdp['GeoFips'], merged_us_gdp['GeoName'], merged_us_gdp['Description'],
     merged_us_gdp['city'])


In [10]:
merged_us_gdp.head()

,LineCode,2011,2012,2013,airport
0,1.0,277405509.0,287570424.0,299918186.0,ATL
1,2.0,250779823.0,260962774.0,273113553.0,ATL
2,3.0,NaN,NaN,602096.0,ATL
3,6.0,267771.0,228417.0,231368.0,ATL
4,10.0,3704264.0,3582854.0,NaN,ATL


In [11]:
#melting data frame to obtain years as a factor
melted_us_gdp = merged_us_gdp.melt(id_vars=['LineCode', 'airport'], 
                                   value_vars=['2011', '2012', '2013'],
                                   var_name='year', value_name='gdp')
melted_us_gdp.head()

,LineCode,airport,year,gdp
0,1.0,ATL,2011,277405509.0
1,2.0,ATL,2011,250779823.0
2,3.0,ATL,2011,NaN
3,6.0,ATL,2011,267771.0
4,10.0,ATL,2011,3704264.0


In [12]:
#converting dates to datetime
melted_us_gdp['year'] = pd.to_datetime(melted_us_gdp['year'], format='%Y')
melted_us_gdp.set_index('year', inplace=True)
melted_us_gdp.head()

,LineCode,airport,gdp
year,,,
2011-01-01,1.0,ATL,277405509.0
2011-01-01,2.0,ATL,250779823.0
2011-01-01,3.0,ATL,NaN
2011-01-01,6.0,ATL,267771.0
2011-01-01,10.0,ATL,3704264.0


In [19]:
melted_us_gdp.to_csv(os.path.join('processed_add_data', 'gdp_data_processed.csv'))

## US holiday data

In [71]:
us_hol = pd.read_csv(os.path.join('raw_add_data', 'usholidays.csv'), usecols=['Date', 'Holiday'])
us_hol['Date'] = pd.to_datetime(us_hol['Date']) # convert dates to datetime
us_hol.set_index('Date', inplace=True)
us_hol.head()

,Holiday
Date,
2010-12-31,New Year's Day
2011-01-17,"Birthday of Martin Luther King, Jr."
2011-02-21,Washington's Birthday
2011-05-30,Memorial Day
2011-07-04,Independence Day


To select the relevant dates, we rely on the train and test datasets

In [18]:
x_train, _ = problem.get_train_data()
x_test, _ = problem.get_test_data()

max_date = max(x_test['DateOfDeparture'].max(), x_train['DateOfDeparture'].max())
min_date = min(x_test['DateOfDeparture'].min(), x_train['DateOfDeparture'].min())

print(f'The max date is: {max_date}')
print(f'The min date is: {min_date}')

The max date is: 2013-03-05
The min date is: 2011-09-01


In [21]:
us_hol = us_hol[min_date : max_date]
us_hol.to_csv(os.path.join('processed_add_data', 'usholidays_processed.csv'))

## US flights data

The data comes from the Bureau of Transportation Statistics' T-100 form. It aggregates the domestic flight infomartion monthly by Air Company in the US. 

In [72]:
flight_11 = pd.read_csv(os.path.join('raw_add_data', 'flights_data_2011.csv'))
flight_12 = pd.read_csv(os.path.join('raw_add_data', 'flights_data_2012.csv'))
flight_13 = pd.read_csv(os.path.join('raw_add_data', 'flights_data_2013.csv'))

In [73]:
for dataset, year in zip([flight_11, flight_12, flight_13], [2011, 2012, 2013]):
    dataset['YEAR'] = year
flight = pd.concat([flight_11, flight_12, flight_13])

### Data exploration

In [74]:
print(flight.columns)

Index(['DEPARTURES_SCHEDULED', 'DEPARTURES_PERFORMED', 'PAYLOAD', 'SEATS',
       'PASSENGERS', 'FREIGHT', 'MAIL', 'DISTANCE', 'RAMP_TO_RAMP', 'AIR_TIME',
       'CARRIER_NAME', 'ORIGIN', 'DEST', 'MONTH', 'Unnamed: 14', 'YEAR'],
      dtype='object')


In [75]:
print(flight['Unnamed: 14'].unique()) #irrelevant column
del flight['Unnamed: 14']

[nan]


We are not interested in the `PAYLOAD`, `FREIGHT`, `MAIL`, `RAMP_TO_RAMP`, `AIR_TIME`, and `CARRIER_NAME` variables

In [76]:
del (flight['PAYLOAD'], flight['FREIGHT'], flight['MAIL'], flight['RAMP_TO_RAMP'],
     flight['AIR_TIME'], flight['CARRIER_NAME'])

In [77]:
print(flight.columns)

Index(['DEPARTURES_SCHEDULED', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS',
       'DISTANCE', 'ORIGIN', 'DEST', 'MONTH', 'YEAR'],
      dtype='object')


In [78]:
flight.head()

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,DISTANCE,ORIGIN,DEST,MONTH,YEAR
0,0.0,1.0,30.0,27.0,385.0,ACY,CRW,9,2011
1,0.0,16.0,480.0,332.0,107.0,ACY,FRG,9,2011
2,0.0,1.0,30.0,15.0,120.0,ACY,HPN,9,2011
3,0.0,13.0,390.0,319.0,121.0,ACY,ISP,9,2011
4,0.0,1.0,30.0,28.0,263.0,ACY,LBE,9,2011


In [81]:
test1 = flight['ORIGIN']=='LAX'
test2 = flight['DEST']=='JFK'
test3 = flight['MONTH']==3
test4 = flight['YEAR']==2011
flight.loc[test1 & test2 & test3 & test4, :]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,DISTANCE,ORIGIN,DEST,MONTH,YEAR
4071,0.0,26.0,6096.0,3978.0,2475.0,LAX,JFK,3,2011
4092,0.0,1.0,0.0,0.0,2475.0,LAX,JFK,3,2011
121876,1.0,1.0,225.0,133.0,2475.0,LAX,JFK,3,2011
160335,2.0,2.0,376.0,355.0,2475.0,LAX,JFK,3,2011
327561,89.0,90.0,13500.0,11681.0,2475.0,LAX,JFK,3,2011
348647,181.0,181.0,26921.0,23399.0,2475.0,LAX,JFK,3,2011
348673,182.0,182.0,20020.0,17479.0,2475.0,LAX,JFK,3,2011
350191,209.0,209.0,36361.0,29610.0,2475.0,LAX,JFK,3,2011
351999,302.0,300.0,50400.0,43643.0,2475.0,LAX,JFK,3,2011


Since our original data was divided by air carrier, we have more than one entry per month. We need, therefore, to group the data and sum the columns `DEPARTURES_SCHEDULED`, `DEPARTURES_PERFORMED`, `SEATS`, and `PASSENGERS`

In [82]:
flight = flight.groupby(['MONTH', 'YEAR', 'ORIGIN', 'DEST', 'DISTANCE'], as_index=False).sum()

In [86]:
#check that we have only one entry per month
# redefine tests as indexes were changed
test1 = flight['ORIGIN']=='LAX'
test2 = flight['DEST']=='JFK'
test3 = flight['MONTH']==3
test4 = flight['YEAR']==2011
flight.loc[test1 & test2 & test3 & test4, :]

,MONTH,YEAR,ORIGIN,DEST,DISTANCE,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS
66042,3,2011,LAX,JFK,2475.0,966.0,992.0,153899.0,130278.0


### Data Transformation

We want to calculate the total number of passengers, flights, and seats that originated from each of the airports.

In [101]:
origin_totals = flight.loc[:, flight.columns != 'DISTANCE'].groupby(
    ['MONTH', 'YEAR', 'ORIGIN'], as_index=False
                                                                    ).sum()

In [103]:
origin_totals.loc[origin_totals['ORIGIN']=='LAX']

,MONTH,YEAR,ORIGIN,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS
429,1,2011,LAX,18248.0,18317.0,2224646.0,1676392.0
1248,1,2012,LAX,18822.0,19165.0,2257668.0,1773288.0
2055,1,2013,LAX,18585.0,18776.0,2260225.0,1762373.0
2879,2,2011,LAX,16552.0,16677.0,2006175.0,1529573.0
3698,2,2012,LAX,18015.0,18201.0,2129305.0,1670438.0
4512,2,2013,LAX,16894.0,17104.0,2061814.0,1627773.0
5354,3,2011,LAX,19164.0,19412.0,2326791.0,1884726.0
6190,3,2012,LAX,20166.0,20334.0,2394209.0,2008198.0
7004,3,2013,LAX,19556.0,19757.0,2395150.0,2029950.0
7835,4,2011,LAX,19214.0,19405.0,2291370.0,1883865.0
